In [2]:
import sys
sys.path.append('..')
wp = '/home/jose/lung_cancer_ds_bowl/'

In [6]:
import os
import random
import logging
import multiprocessing
import numpy as np
import pandas as pd
from time import time
import matplotlib.pyplot as plt
from utils import plotting
from dl_model_pos_patches import  common
from sklearn import metrics
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback
from dl_networks.sample_resnet import ResnetBuilder
from dl_utils.tb_callback import TensorBoard

In [8]:
INPUT_PATH = '/mnt/hd2/preprocessed5'
VALIDATION_PATH = '/mnt/hd2/preprocessed5_validation_luna'
NODULES_PATH = wp + 'data/luna/annotations.csv'
PATCHES_PATH = '/mnt/hd2/patches'

K.set_image_dim_ordering('th')

In [13]:
### PATCHES GENERATION -----------------------------------------------------------------

## PATIENTS FILE LIST
patients_with_annotations = pd.read_csv(NODULES_PATH)  # filter patients with no annotations to avoid having to read them
patients_with_annotations = list(set(patients_with_annotations['seriesuid']))
patients_with_annotations = ["luna_%s.npz" % p.split('.')[-1] for p in patients_with_annotations]

filenames = os.listdir(INPUT_PATH)
filenames = [g for g in filenames if g.startswith('luna_')]
filenames_train = [os.path.join(INPUT_PATH, fp) for fp in filenames if fp in patients_with_annotations]
filenames_test = [os.path.join(VALIDATION_PATH, fp) for fp in os.listdir(VALIDATION_PATH) if fp in patients_with_annotations]
filenames_dsb = [os.path.join(INPUT_PATH, g) for g in filenames if g.startswith('dsb_')]

In [11]:
def __load_and_store(filename):
    patient_data = np.load(filename)['arr_0']
    X, y, rois, stats = common.load_patient_3d_atlas(patient_data, 
                                                     discard_empty_nodules=True, output_rois=True, debug=True, thickness=1
                                                    )
    #logging.info("Patient: %s, stats: %s" % (filename.split('/')[-1], stats))
    return X, y, stats

In [ ]:
for filepath in filenames_train[0]:
    common.multiproc_crop_generator(filepath,
                                os.path.join(PATCHES_PATH,(filepath.split('/')[-1]+'_x.npz')),
                                os.path.join(PATCHES_PATH,(filepath.split('/')[-1]+'_y.npz')),
                                __load_and_store, store=True)